## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-08-11-52-14 +0000,bbc,At least 24 killed as army paraglider bombs My...,https://www.bbc.com/news/articles/cge2l1xj2zdo...
1,2025-10-08-11-50-36 +0000,nypost,Maniac threatened to detonate tent filled with...,https://nypost.com/2025/10/08/us-news/maniac-t...
2,2025-10-08-11-46-57 +0000,bbc,Have you been mis-sold car finance? Here's wha...,https://www.bbc.com/news/videos/c4g51p321ggo?a...
3,2025-10-08-11-45-09 +0000,bbc,"UK won't relax visa rules for India, Starmer says",https://www.bbc.com/news/articles/c9wdzryk477o...
4,2025-10-08-11-40-00 +0000,wsj,Trio Wins Nobel Prize in Chemistry for Work on...,https://www.wsj.com/science/nobel-chemistry-pr...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2386/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
353,trump,33
167,new,21
94,gaza,15
406,shutdown,14
19,court,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
262,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...,86
44,2025-10-08-10-00-00 +0000,nypost,Meet the Americans priced out of prescription ...,https://nypost.com/2025/10/08/health/americans...,85
270,2025-10-07-17-41-13 +0000,nypost,Trump says Dems ‘have no leader’ in shutdown s...,https://nypost.com/2025/10/07/us-news/trump-ri...,84
118,2025-10-08-03-26-15 +0000,nyt,Trump Suggests Furloughed Federal Workers May ...,https://www.nytimes.com/2025/10/07/us/politics...,78
161,2025-10-07-23-53-21 +0000,nypost,Mother of US Navy medic rescued from Gaza in d...,https://nypost.com/2025/10/07/world-news/mothe...,75


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
262,86,2025-10-07-18-11-39 +0000,nypost,Trump Middle East envoy won’t leave Egypt unti...,https://nypost.com/2025/10/07/world-news/kushn...
58,60,2025-10-08-09-30-00 +0000,wsj,"In Japan, as in the U.S., a new leader wants t...",https://www.wsj.com/economy/central-banking/go...
302,58,2025-10-07-15-50-55 +0000,nypost,NY woman refuses to pay $440 fine to Hertz aft...,https://nypost.com/2025/10/07/business/ny-woma...
315,52,2025-10-07-15-03-14 +0000,nypost,Nobel winner Fred Ramsdell didn’t know he’d wo...,https://nypost.com/2025/10/07/world-news/nobel...
282,50,2025-10-07-17-01-48 +0000,latimes,Supreme Court sees a free-speech problem with ...,https://www.latimes.com/politics/story/2025-10...
225,47,2025-10-07-20-45-00 +0000,wsj,White House Says Federal Workers’ Back Pay Dur...,https://www.wsj.com/politics/policy/customurl-...
299,42,2025-10-07-16-03-10 +0000,nypost,Pam Bondi lashes out at Dems in rowdy hearing ...,https://nypost.com/2025/10/07/us-news/pam-bond...
234,40,2025-10-07-20-26-13 +0000,nypost,Connecticut governor warns Wall Street at risk...,https://nypost.com/2025/10/07/business/connect...
124,39,2025-10-08-02-46-58 +0000,nyt,Israel Marks Two-Year Anniversary of Oct. 7 in...,https://www.nytimes.com/2025/10/07/world/middl...
149,37,2025-10-08-00-44-44 +0000,nyt,Athens Democracy Forum: As Democracy Erodes Gl...,https://www.nytimes.com/2025/10/07/world/europ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
